In [ ]:
download_dir = r"C:\Wetland Tracker 2025"  # Update this to your desired download folder


#### HTML code for the eye button

https://claude.ai/chat/bd56db03-5b92-4037-b346-70ca1dc8986a

<span data-testid="tooltipWrapper" class="usa-tooltip"><button aria-describedby="tooltip-956487" class="usa-button usa-tooltip__trigger view-notice" data-testid="triggerElement" tabindex="0" title="" type="button"><svg viewBox="0 0 24 24" role="img" focusable="false" tabindex="-1" style="width: 16px; height: 16px;"><path d="M12,9A3,3 0 0,0 9,12A3,3 0 0,0 12,15A3,3 0 0,0 15,12A3,3 0 0,0 12,9M12,17A5,5 0 0,1 7,12A5,5 0 0,1 12,7A5,5 0 0,1 17,12A5,5 0 0,1 12,17M12,4.5C7,4.5 2.73,7.61 1,12C2.73,16.39 7,19.5 12,19.5C17,19.5 21.27,16.39 23,12C21.27,7.61 17,4.5 12,4.5Z" style="fill: currentcolor;"></path></svg></button></span>

In [ ]:
from helium import set_driver, click, find_all, S, kill_browser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import os
from urllib.parse import urlparse, unquote

# Specify your download directory
download_dir = r"C:\Wetland Tracker 2025\rrs2"  # Update this to your desired download folder
os.makedirs(download_dir, exist_ok=True)  # Ensure the directory exists

# Create Chrome options for automatic PDF download
chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True  # Force PDFs to download
}
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.add_argument("--disable-extensions")  # Disable extensions to prevent interference

# Create a Chrome driver with the custom options and set it for Helium
driver = webdriver.Chrome(options=chrome_options)
set_driver(driver)

# Function to generate a unique filename from URL
def get_filename_from_url(url, index):
    # Parse the URL and get the path
    parsed_url = urlparse(url)
    path = unquote(parsed_url.path)
    
    # Extract the filename from the path
    filename = os.path.basename(path)
    
    # If no filename is extracted, use a default name with the index
    if not filename or not filename.endswith('.pdf'):
        filename = f"notice_{index+1}.pdf"
    
    # To ensure uniqueness, prefix with the index
    unique_filename = f"{index+1}_{filename}"
    
    return unique_filename

# Track downloaded URLs to avoid duplicates
downloaded_urls = set()

# Open the target URL and wait for it to load
driver.get("https://rrs.usace.army.mil/rrs/public-notices")
print("Loaded initial page")
time.sleep(5)  # Longer initial wait

# Accept the consent pop-up by clicking "I Accept"
try:
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'I Accept')]"))
    )
    accept_button.click()
    print("Clicked 'I Accept'")
except Exception as e:
    print(f"Could not click 'I Accept': {str(e)}")
time.sleep(3)

# Click the "Table View" button to display the table
try:
    table_view_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Table View')]"))
    )
    table_view_button.click()
    print("Clicked 'Table View'")
except Exception as e:
    print(f"Could not click 'Table View': {str(e)}")
time.sleep(3)

# Get the table rows to extract notice information
try:
    table_rows = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table.usa-table tbody tr"))
    )
    print(f"Found {len(table_rows)} table rows")
except Exception as e:
    print(f"Error finding table rows: {str(e)}")
    table_rows = []

# Locate all the eye buttons using Selenium directly
eye_buttons = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "button.view-notice"))
)
print(f"Found {len(eye_buttons)} eye buttons")

# Iterate over each eye button to click and download the PDF
for index, btn in enumerate(eye_buttons):
    print(f"\nProcessing eye button {index+1}/{len(eye_buttons)}")
    try:
        # Get additional information from the row for filename if available
        notice_id = f"notice_{index+1}"
        try:
            if index < len(table_rows):
                row = table_rows[index]
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) >= 2:  # Assuming ID is in the first or second column
                    notice_id = cells[0].text.strip() or cells[1].text.strip()
                    notice_id = re.sub(r'[\\/*?:"<>|]', "_", notice_id)  # Remove invalid filename chars
        except Exception as e:
            print(f"Error getting row data: {str(e)}")
        
        # Scroll the button into view
        driver.execute_script("arguments[0].scrollIntoView(true);", btn)
        time.sleep(1)
        
        # Click the button using JavaScript for more reliability
        driver.execute_script("arguments[0].click();", btn)
        print(f"Clicked eye button {index+1}")
        time.sleep(3)  # Allow time for the popup to appear
        
        # Clear all open tabs except the first one
        while len(driver.window_handles) > 1:
            driver.switch_to.window(driver.window_handles[1])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        
        # Try to find any PDF link element in the popup
        try:
            link_elements = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, '.pdf')]"))
            )
            print(f"Found {len(link_elements)} PDF links in popup")
            
            for i, link in enumerate(link_elements):
                href = link.get_attribute("href")
                
                # Skip if we've already downloaded this URL
                if href in downloaded_urls:
                    print(f"Skipping already downloaded URL: {href}")
                    continue
                
                print(f"Link {i+1} href: {href}")
                
                # Extract actual URL from extension prefix if present
                actual_url = href
                if "extension://" in href:
                    match = re.search(r'extension://[^/]+/(https?://[^"]+)', href)
                    if match:
                        actual_url = match.group(1)
                        print(f"Extracted actual URL: {actual_url}")
                
                # Get a unique filename for this download
                filename = get_filename_from_url(actual_url, index)
                
                # Get the existing files before download
                before_files = set(os.listdir(download_dir))
                
                # Open URL in new tab to download
                print(f"Opening URL in new tab: {actual_url}")
                driver.execute_script(f"window.open('{actual_url}');")
                
                # Add this URL to our downloaded set
                downloaded_urls.add(href)
                
                # Wait for download to complete (up to 30 seconds)
                download_complete = False
                for _ in range(30):
                    time.sleep(1)
                    current_files = set(os.listdir(download_dir))
                    new_files = current_files - before_files
                    if new_files:
                        print(f"New files detected: {new_files}")
                        # Found new files, download seems complete
                        download_complete = True
                        
                        # Rename the downloaded file if we can identify it
                        if len(new_files) == 1:
                            downloaded_file = os.path.join(download_dir, list(new_files)[0])
                            new_file_path = os.path.join(download_dir, f"{notice_id}_{filename}")
                            try:
                                # Wait a bit to ensure file is fully written
                                time.sleep(1)
                                os.rename(downloaded_file, new_file_path)
                                print(f"Renamed downloaded file to: {new_file_path}")
                            except Exception as e:
                                print(f"Error renaming file: {str(e)}")
                        break
                
                if not download_complete:
                    print("Warning: Download may not have completed within the timeout period")
                
                # Close all tabs except the first one
                while len(driver.window_handles) > 1:
                    driver.switch_to.window(driver.window_handles[1])
                    driver.close()
                driver.switch_to.window(driver.window_handles[0])
                print("Switched back to original tab")
                break  # Just process the first PDF link we find
                
        except Exception as e:
            print(f"Error finding or processing PDF links: {str(e)}")
        
        # Close the popup (try multiple methods)
        try:
            # Try to find a close button
            close_buttons = driver.find_elements(By.XPATH, "//button[contains(@class, 'close') or @aria-label='Close' or contains(text(), 'Close')]")
            if close_buttons:
                driver.execute_script("arguments[0].click();", close_buttons[0])
                print("Clicked close button")
            else:
                # Try to find a clickable overlay/backdrop
                backdrops = driver.find_elements(By.CSS_SELECTOR, ".modal-backdrop, .overlay, .usa-modal__overlay")
                if backdrops:
                    driver.execute_script("arguments[0].click();", backdrops[0])
                    print("Clicked modal backdrop")
                else:
                    # Press Escape key as last resort
                    from selenium.webdriver.common.keys import Keys
                    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                    print("Pressed Escape key")
        except Exception as e:
            print(f"Error closing popup: {str(e)}")
        
        time.sleep(2)  # Wait between items
    
    except Exception as e:
        print(f"Error processing button {index+1}: {str(e)}")

print("Script completed")
# Close the browser when done
kill_browser()

## Working but only PDFs

next steps:

- Get CSV once

- At same time, paginate through all and get all Eye popup buttons

- Then a few days later, go to the website and get the latest CSV - compare with the old one. 

- Filter to rows which are NOT in the new one.

- Then only navigate to the eye buttons for those rows (or get all eye buttons and open only those which are new (LEFT ANTI JOIN))

In [1]:
from helium import set_driver, click, find_all, S, kill_browser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import os
from urllib.parse import urlparse, unquote

# Specify your download directory
download_dir = r"C:\Wetland Tracker 2025\rrs_test"  # Update this to your desired download folder
os.makedirs(download_dir, exist_ok=True)  # Ensure the directory exists

# Create Chrome options for automatic PDF download
chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True  # Force PDFs to download
}
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.add_argument("--disable-extensions")  # Disable extensions to prevent interference

# Create a Chrome driver with the custom options and set it for Helium
driver = webdriver.Chrome(options=chrome_options)
set_driver(driver)

# Function to generate a unique filename from URL
def get_filename_from_url(url, index):
    # Parse the URL and get the path
    parsed_url = urlparse(url)
    path = unquote(parsed_url.path)
    
    # Extract the filename from the path
    filename = os.path.basename(path)
    
    # If no filename is extracted, use a default name with the index
    if not filename or not filename.endswith('.pdf'):
        filename = f"notice_{index+1}.pdf"
    
    # To ensure uniqueness, prefix with the index
    unique_filename = f"{index+1}_{filename}"
    
    return unique_filename

# Track downloaded URLs to avoid duplicates
downloaded_urls = set()

# Open the target URL and wait for it to load
driver.get("https://rrs.usace.army.mil/rrs/public-notices")
print("Loaded initial page")
time.sleep(5)  # Longer initial wait

# Accept the consent pop-up by clicking "I Accept"
try:
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'I Accept')]"))
    )
    accept_button.click()
    print("Clicked 'I Accept'")
except Exception as e:
    print(f"Could not click 'I Accept': {str(e)}")
time.sleep(3)

# Click the "Table View" button to display the table
try:
    table_view_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Table View')]"))
    )
    table_view_button.click()
    print("Clicked 'Table View'")
except Exception as e:
    print(f"Could not click 'Table View': {str(e)}")
time.sleep(3)

# Get the table rows to extract notice information
try:
    table_rows = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table.usa-table tbody tr"))
    )
    print(f"Found {len(table_rows)} table rows")
except Exception as e:
    print(f"Error finding table rows: {str(e)}")
    table_rows = []

# Locate all the eye buttons using Selenium directly
eye_buttons = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "button.view-notice"))
)
print(f"Found {len(eye_buttons)} eye buttons")

# Iterate over each eye button to click and download the PDF
for index, btn in enumerate(eye_buttons):
    print(f"\nProcessing eye button {index+1}/{len(eye_buttons)}")
    try:
        # Get additional information from the row for filename if available
        notice_id = f"notice_{index+1}"
        try:
            if index < len(table_rows):
                row = table_rows[index]
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) >= 2:  # Assuming ID is in the first or second column
                    notice_id = cells[0].text.strip() or cells[1].text.strip()
                    notice_id = re.sub(r'[\\/*?:"<>|]', "_", notice_id)  # Remove invalid filename chars
        except Exception as e:
            print(f"Error getting row data: {str(e)}")
        
        # Scroll the button into view
        driver.execute_script("arguments[0].scrollIntoView(true);", btn)
        time.sleep(1)
        
        # Click the button using JavaScript for more reliability
        driver.execute_script("arguments[0].click();", btn)
        print(f"Clicked eye button {index+1}")
        time.sleep(3)  # Allow time for the popup to appear
        
        # Clear all open tabs except the first one
        while len(driver.window_handles) > 1:
            driver.switch_to.window(driver.window_handles[1])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        
        # Try to find any PDF link element in the popup
        try:
            link_elements = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, '.pdf')]"))
            )
            print(f"Found {len(link_elements)} PDF links in popup")
            
            for i, link in enumerate(link_elements):
                href = link.get_attribute("href")
                
                # Skip if we've already downloaded this URL
                if href in downloaded_urls:
                    print(f"Skipping already downloaded URL: {href}")
                    continue
                
                print(f"Link {i+1} href: {href}")
                
                # Extract actual URL from extension prefix if present
                actual_url = href
                if "extension://" in href:
                    match = re.search(r'extension://[^/]+/(https?://[^"]+)', href)
                    if match:
                        actual_url = match.group(1)
                        print(f"Extracted actual URL: {actual_url}")
                
                # Get a unique filename for this download
                filename = get_filename_from_url(actual_url, index)
                
                # Get the existing files before download
                before_files = set(os.listdir(download_dir))
                
                # Open URL in new tab to download
                print(f"Opening URL in new tab: {actual_url}")
                driver.execute_script(f"window.open('{actual_url}');")
                
                # Add this URL to our downloaded set
                downloaded_urls.add(href)
                
                # Wait for download to complete (up to 30 seconds)
                download_complete = False
                for _ in range(30):
                    time.sleep(1)
                    current_files = set(os.listdir(download_dir))
                    new_files = current_files - before_files
                    if new_files:
                        print(f"New files detected: {new_files}")
                        # Found new files, download seems complete
                        download_complete = True
                        
                        # Rename the downloaded file if we can identify it
                        if len(new_files) == 1:
                            downloaded_file = os.path.join(download_dir, list(new_files)[0])
                            new_file_path = os.path.join(download_dir, f"{notice_id}_{filename}")
                            try:
                                # Wait a bit to ensure file is fully written
                                time.sleep(1)
                                os.rename(downloaded_file, new_file_path)
                                print(f"Renamed downloaded file to: {new_file_path}")
                            except Exception as e:
                                print(f"Error renaming file: {str(e)}")
                        break
                
                if not download_complete:
                    print("Warning: Download may not have completed within the timeout period")
                
                # Close all tabs except the first one
                while len(driver.window_handles) > 1:
                    driver.switch_to.window(driver.window_handles[1])
                    driver.close()
                driver.switch_to.window(driver.window_handles[0])
                print("Switched back to original tab")
                break  # Just process the first PDF link we find
                
        except Exception as e:
            print(f"Error finding or processing PDF links: {str(e)}")
        
        # Close the popup using the exact selector for the close button
        try:
            # Target the specific close button with the exact attributes you provided
            close_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button.usa-button.usa-modal__close[aria-label='Close this window'][data-close-modal='true']"))
            )
            driver.execute_script("arguments[0].click();", close_button)
            print("Clicked the exact close button")
        except Exception as e:
            print(f"Error with specific close button: {str(e)}")
            # Fallback method if the specific selector fails
            try:
                # Try any close button with matching attributes
                close_buttons = driver.find_elements(By.XPATH, "//button[contains(@class, 'usa-modal__close') or @aria-label='Close this window' or @data-close-modal='true']")
                if close_buttons:
                    driver.execute_script("arguments[0].click();", close_buttons[0])
                    print("Clicked alternative close button")
                else:
                    # Press Escape key as last resort
                    from selenium.webdriver.common.keys import Keys
                    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                    print("Pressed Escape key")
            except Exception as e2:
                print(f"All close attempts failed: {str(e2)}")
        
        time.sleep(2)  # Wait between items
    
    except Exception as e:
        print(f"Error processing button {index+1}: {str(e)}")

print("Script completed")
# Close the browser when done
kill_browser()

Loaded initial page
Clicked 'I Accept'
Could not click 'Table View': Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF709195305+78597]
	GetHandleVerifier [0x00007FF709195360+78688]
	(No symbol) [0x00007FF708F491AA]
	(No symbol) [0x00007FF708F9F149]
	(No symbol) [0x00007FF708F9F3FC]
	(No symbol) [0x00007FF708FF2467]
	(No symbol) [0x00007FF708FC712F]
	(No symbol) [0x00007FF708FEF2BB]
	(No symbol) [0x00007FF708FC6EC3]
	(No symbol) [0x00007FF708F903F8]
	(No symbol) [0x00007FF708F91163]
	GetHandleVerifier [0x00007FF70943EEBD+2870973]
	GetHandleVerifier [0x00007FF709439668+2848360]
	GetHandleVerifier [0x00007FF709456943+2967875]
	GetHandleVerifier [0x00007FF7091B014A+188746]
	GetHandleVerifier [0x00007FF7091B842F+222255]
	GetHandleVerifier [0x00007FF70919D284+111236]
	GetHandleVerifier [0x00007FF70919D432+111666]
	GetHandleVerifier [0x00007FF709183559+5465]
	BaseThreadInitThunk [0x00007FFADD97E8D7+23]
	RtlUserThreadStart [0x00007FFADEE714FC+44]

Found 10 table rows
Found 10 eye buttons

Pr

KeyboardInterrupt: 

In [ ]:
# ! pip install 
import subprocess
import sys
python_executable = sys.executable

subprocess.check_call([python_executable, "-m", "pip", "install", "webdriver-manager"])

In [3]:
import nest_asyncio
nest_asyncio.apply()

## Step 1: Get latest CSV

In [ ]:
## inspect 'Export CSV' button 

# <button type="button" class="usa-button action-button button-small loading-button" data-testid="button" title="Export CSV"><svg viewBox="0 0 24 24" role="img" focusable="false" tabindex="-1" style="width: 1.5rem; height: 1.5rem;"><path d="M6,2C4.89,2 4,2.9 4,4V20A2,2 0 0,0 6,22H18A2,2 0 0,0 20,20V8L14,2M13,3.5L18.5,9H13M8.93,12.22H16V19.29L13.88,17.17L11.05,20L8.22,17.17L11.05,14.35" style="fill: currentcolor;"></path></svg>Export CSV</button>

# <svg viewBox="0 0 24 24" role="img" focusable="false" tabindex="-1" style="width: 1.5rem; height: 1.5rem;"><path d="M6,2C4.89,2 4,2.9 4,4V20A2,2 0 0,0 6,22H18A2,2 0 0,0 20,20V8L14,2M13,3.5L18.5,9H13M8.93,12.22H16V19.29L13.88,17.17L11.05,20L8.22,17.17L11.05,14.35" style="fill: currentcolor;"></path></svg> "Export CSV"



In [8]:
import os
import pandas as pd

# scrape and export CSV


# set directory
os.chdir(r"C:\Wetland Tracker 2025\rrs")


# read in new file
df = pd.read_csv(r"C:\Wetland Tracker 2025\rrs\PublicNotices (2).csv")

# read in old file

dfo = pd.read_csv(r"C:\Wetland Tracker 2025\rrs\PublicNotices (2).csv")
dfo = dfo.iloc[2:, :]


In [16]:

df_new = df.merge(dfo, how='outer', on = ['DA Number', 'Date of Public Notice', 'District'], indicator=True).loc[lambda x : x['_merge']=='left_only']

df_new

,DA Number,Project Name_x,Description_x,Type_x,Date of Public Notice,Comment Period Ends_x,Applicant Name_x,Applicant Company_x,Keywords_x,District,...,Applicant Name_y,Applicant Company_y,Keywords_y,State_y,County_y,Zip Code_y,Hydrologic Unit Code_y,Latitude_y,Longitude_y,_merge
24,NAB-2022-00093-PO2,PA DOT - Fetter Advanced Wetland Compensation ...,"United States Army Corps of Engineers, Baltimo...",DEVMBA,03/14/2025 00:00:00,04/13/2025 00:00:00,Drew Ames,PA DOT,"permit, NAB-2022-00093-P02, PA DOT, Fetter Adv...",Baltimore,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
131,SWT-2022-00563,Multi-Purpose Structure Fourche Maline Creek N...,The rehabilitation project would bring Fourche...,SP,03/20/2025 00:00:00,04/18/2025 00:00:00,Jim Grego,Latimer County Conservation District,"Public Notice, U.S. Army Corps of Engineers, T...",Tulsa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [17]:
print(df_new)

              DA Number                                     Project Name_x  \
24   NAB-2022-00093-PO2  PA DOT - Fetter Advanced Wetland Compensation ...   
131      SWT-2022-00563  Multi-Purpose Structure Fourche Maline Creek N...   

                                         Description_x  Type_x  \
24   United States Army Corps of Engineers, Baltimo...  DEVMBA   
131  The rehabilitation project would bring Fourche...      SP   

    Date of Public Notice Comment Period Ends_x Applicant Name_x  \
24    03/14/2025 00:00:00   04/13/2025 00:00:00        Drew Ames   
131   03/20/2025 00:00:00   04/18/2025 00:00:00        Jim Grego   

                      Applicant Company_x  \
24                                PA DOT    
131  Latimer County Conservation District   

                                            Keywords_x   District  ...  \
24   permit, NAB-2022-00093-P02, PA DOT, Fetter Adv...  Baltimore  ...   
131  Public Notice, U.S. Army Corps of Engineers, T...      Tulsa  ...   

  

## Export CSV

In [ ]:
import random
import time
from selenium.webdriver import ActionChains

def human_click(driver, element):
    """Perform a more 'human-like' hover & click on the given WebElement."""
    actions = ActionChains(driver)
    
    # Move the mouse over the element, pause briefly for realism
    actions.move_to_element(element).pause(random.uniform(0.5, 1.5))
    
    # Click the element
    actions.click(element)
    actions.perform()
    
    # After-click pause
    time.sleep(random.uniform(1, 2))


In [6]:
from helium import set_driver, kill_browser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import pandas as pd

download_dir = r"C:\Wetland Tracker 2025\rrs2"
os.makedirs(download_dir, exist_ok=True)

chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True
}
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.add_argument("--disable-extensions")

driver = webdriver.Chrome(options=chrome_options)
set_driver(driver)

try:
    driver.get("https://rrs.usace.army.mil/rrs/public-notices")
    print("Loaded initial page")
    time.sleep(5)

    # 1) Accept consent pop-up
    try:
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'I Accept')]"))
        )
        accept_button.click()
        print("Clicked 'I Accept'")
    except Exception as e:
        print(f"Could not click 'I Accept': {str(e)}")

    time.sleep(3)

    # 2) Click "Table View" button if needed
    try:
        table_view_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Table View')]"))
        )
        table_view_button.click()
        print("Clicked 'Table View'")
    except Exception as e:
        print(f"Could not click 'Table View': {str(e)}")

    time.sleep(3)

    # 3) Locate the "Export CSV" button, scroll into view, and click via JS
    try:
        csv_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, "//button[contains(text(), 'Export CSV')]")
            )
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", csv_button)
        time.sleep(1)  # give it a moment in case a floating element needs to animate away

        # If you know the floating button that’s in the way, you can try to close it or hide it:
        # Example:
        # floating_button = driver.find_element(By.CSS_SELECTOR, "button.usa-button.float")
        # if floating_button:
        #     floating_button.click()  # or do something else to remove/hide it

        # Finally, do a JS-based click to avoid “element click intercepted”
        driver.execute_script("arguments[0].click();", csv_button)
        print("Clicked 'Export CSV' button (via JavaScript)")
    except Exception as e:
        print(f"Could not click 'Export CSV': {e}")

    # Wait for the CSV file to download
    before_files = set(os.listdir(download_dir))
    new_file_name = None
    for _ in range(30):
        time.sleep(1)
        after_files = set(os.listdir(download_dir))
        new_files = after_files - before_files
        if new_files:
            for f in new_files:
                if f.lower().endswith('.csv'):
                    new_file_name = os.path.join(download_dir, f)
                    print(f"Detected new CSV file: {new_file_name}")
                    break
            if new_file_name:
                break

    if not new_file_name:
        print("No new CSV file was detected in time.")
    else:
        df = pd.read_csv(new_file_name)
        print(f"\nDataFrame loaded from {new_file_name}:\n")
        print(df.head())

finally:
    print("Closing browser.")
    kill_browser()


Loaded initial page
Clicked 'I Accept'
Could not click 'Table View': Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7F87A4C25+3179557]
	(No symbol) [0x00007FF7F84088A0]
	(No symbol) [0x00007FF7F82991CA]
	(No symbol) [0x00007FF7F82EFA67]
	(No symbol) [0x00007FF7F82EFC9C]
	(No symbol) [0x00007FF7F8343627]
	(No symbol) [0x00007FF7F8317C6F]
	(No symbol) [0x00007FF7F83402F3]
	(No symbol) [0x00007FF7F8317A03]
	(No symbol) [0x00007FF7F82E06D0]
	(No symbol) [0x00007FF7F82E1983]
	GetHandleVerifier [0x00007FF7F88067CD+3579853]
	GetHandleVerifier [0x00007FF7F881D1D2+3672530]
	GetHandleVerifier [0x00007FF7F8812153+3627347]
	GetHandleVerifier [0x00007FF7F857092A+868650]
	(No symbol) [0x00007FF7F8412FFF]
	(No symbol) [0x00007FF7F840F4A4]
	(No symbol) [0x00007FF7F840F646]
	(No symbol) [0x00007FF7F83FEAA9]
	BaseThreadInitThunk [0x00007FFB13FBE8D7+23]
	RtlUserThreadStart [0x00007FFB1427BF6C+44]

Clicked 'Export CSV' button (via JavaScript)
Detected new CSV file: C:\Wetland Tracker 2025\rrs2\PublicN

## Filter to recent notices

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from helium import set_driver, kill_browser
import pandas as pd
import time
import os

# 1) Set up your Chrome driver (without PDF preferences, since we’re just scraping table data)
download_dir = r"C:\Wetland Tracker 2025\rrs2"
os.makedirs(download_dir, exist_ok=True)

chrome_options = webdriver.ChromeOptions()
# (Optional) add other preferences or arguments if needed
driver = webdriver.Chrome(options=chrome_options)
set_driver(driver)

# 2) Go to target URL
driver.get("https://rrs.usace.army.mil/rrs/public-notices")
time.sleep(5)

# 3) Handle initial pop-ups (I Accept, Table View)
try:
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'I Accept')]"))
    )
    accept_button.click()
    print("Clicked 'I Accept'")
except Exception:
    print("No 'I Accept' button found or error clicking it.")

time.sleep(3)

try:
    table_view_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Table View')]"))
    )
    table_view_button.click()
    print("Clicked 'Table View'")
except Exception:
    print("No 'Table View' button found or error clicking it.")

time.sleep(3)

# 4) Determine the total number of pages by locating pagination buttons
pagination_buttons = driver.find_elements(By.XPATH, "//button[@data-testid='pagination-page-number']")
if not pagination_buttons:
    print("No pagination buttons found. Only 1 page to scrape.")
    total_pages = 1
else:
    # The last button typically has the highest page number in its aria-label
    last_label = pagination_buttons[-1].get_attribute('aria-label')  # e.g. "Page 14"
    total_pages = int(last_label.replace("Page ", ""))

print(f"Detected {total_pages} total pages.")

# Prepare a list to hold row data from all pages
all_rows_data = []

# 5) Loop over each page, gather the table data
for page_num in range(1, total_pages + 1):
    print(f"\n--- Navigating to page {page_num} ---")
    
    # If there's more than 1 page, we need to click the page button
    if total_pages > 1:
        try:
            page_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((
                    By.XPATH, f"//button[@data-testid='pagination-page-number' and @aria-label='Page {page_num}']"
                ))
            )
            page_btn.click()
            time.sleep(3)  # Wait for the table to load
        except Exception as e:
            print(f"Could not click page {page_num} button. Error: {e}")
            break

    # Now parse the table on this page
    try:
        table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.pn-table-container table.usa-table"))
        )
        table_rows = table.find_elements(By.CSS_SELECTOR, "tbody tr")
        print(f"Found {len(table_rows)} rows on page {page_num}.")

        for row in table_rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            
            # We expect 7 columns in the table (Actions, DA Number, Project Name, Type,
            #  Date of PN, Date Comment Ends, Applicant)
            # If the structure is stable, we can skip the first cell (Actions) 
            # and then read the next six.
            if len(cells) < 7:
                continue  # skip unexpected row structure

            # Extract the text from each relevant cell
            da_number = cells[1].text.strip()
            project_name = cells[2].text.strip()
            project_type = cells[3].text.strip()
            date_public_notice = cells[4].text.strip()
            date_comment_ends = cells[5].text.strip()
            applicant = cells[6].text.strip()

            row_dict = {
                "DA Number": da_number,
                "Project Name": project_name,
                "Type": project_type,
                "Date of Public Notice": date_public_notice,
                "Date Comment Period Ends": date_comment_ends,
                "Applicant": applicant
            }

            all_rows_data.append(row_dict)

    except Exception as e:
        print(f"Error scraping table on page {page_num}: {e}")

# 6) Convert the accumulated list of dictionaries to a pandas DataFrame
df = pd.DataFrame(all_rows_data)
print("\nScraped table data across all pages:")
print(df)


# 7) Cleanup
kill_browser()

# If you want to explore or manipulate the dataframe programmatically here, 
# you can do so before or after saving.


Clicked 'I Accept'
No 'Table View' button found or error clicking it.
Detected 15 total pages.

--- Navigating to page 1 ---
Found 10 rows on page 1.

--- Navigating to page 2 ---
Found 10 rows on page 2.

--- Navigating to page 3 ---
Found 10 rows on page 3.

--- Navigating to page 4 ---
Found 10 rows on page 4.

--- Navigating to page 5 ---
Found 10 rows on page 5.

--- Navigating to page 6 ---
Found 10 rows on page 6.

--- Navigating to page 7 ---
Found 10 rows on page 7.

--- Navigating to page 8 ---
Found 10 rows on page 8.

--- Navigating to page 9 ---
Found 10 rows on page 9.

--- Navigating to page 10 ---
Found 10 rows on page 10.

--- Navigating to page 11 ---
Found 10 rows on page 11.

--- Navigating to page 12 ---
Found 10 rows on page 12.

--- Navigating to page 13 ---
Found 10 rows on page 13.

--- Navigating to page 14 ---
Found 10 rows on page 14.

--- Navigating to page 15 ---
Found 6 rows on page 15.

Scraped table data across all pages:
              DA Number        

In [9]:
df

,DA Number,Project Name,Type,Date of Public Notice,Date Comment Period Ends,Applicant
0,NAB-2022-00093-PO2,PA DOT - Fetter Advanced Wetland Compensation ...,DEVMBA,03/14/2025,04/13/2025,PA DOT
1,SWT-2022-00563,Multi-Purpose Structure Fourche Maline Creek N...,SP,03/20/2025,04/18/2025,Latimer County Conservation District
2,NAN-2023-00409-ULA,Albany County Airport Authority,SP,03/31/2025,05/01/2025,Albany County Airport Authority
3,MVN-2022-00367-WII,"Wade, West Dock Extension on Bayou Boeuf - St ...",SP,03/17/2025,04/06/2025,Morgan City Harbor and Terminal District
4,SAJ-2023-00531,Shoreline Park South access improvements,SP,03/14/2025,04/13/2025,
...,...,...,...,...,...,...
141,SAC-2023-00293,MCAS Beaufort Fuel Pier Maintenance Dredging,SP,03/31/2025,04/15/2025,MCAS Beaufort
142,NWP-2003-00702,J.R. Simplot Co.- Rivergate Terminal RM 2.4 to...,SP,03/28/2025,04/28/2025,J.R. Simplot Company
143,SAM-2015-01400-JEB,"195, LLC Formerly Twilley Builders, Inc. (Leig...",SP,03/24/2025,04/23/2025,"195, LLC, Twilley Builders, Inc."
144,LRC-2025-00009,Special Public Notices,SPN,03/31/2025,05/31/2025,


## Paginate and download

In [4]:
from helium import set_driver, S, kill_browser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import os
from urllib.parse import urlparse, unquote

# ------------------------------------------------------------------
# 1) Set up your Chrome driver with PDF download preferences
# ------------------------------------------------------------------

download_dir = r"C:\Wetland Tracker 2025\rrs2"
os.makedirs(download_dir, exist_ok=True)

chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
}
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.add_argument("--disable-extensions")

driver = webdriver.Chrome(options=chrome_options)
set_driver(driver)

# ------------------------------------------------------------------
# 2) Open target URL and handle initial pop-ups
# ------------------------------------------------------------------

driver.get("https://rrs.usace.army.mil/rrs/public-notices")
print("Loaded initial page")
time.sleep(5)

# Accept consent pop-up
try:
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'I Accept')]"))
    )
    accept_button.click()
    print("Clicked 'I Accept'")
except Exception as e:
    print(f"Could not click 'I Accept': {str(e)}")

time.sleep(3)

# Click "Table View"
try:
    table_view_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Table View')]"))
    )
    table_view_button.click()
    print("Clicked 'Table View'")
except Exception as e:
    print(f"Could not click 'Table View': {str(e)}")

time.sleep(3)

# ------------------------------------------------------------------
# Helper function to generate a unique filename from URL
# ------------------------------------------------------------------
def get_filename_from_url(url, index):
    parsed_url = urlparse(url)
    path = unquote(parsed_url.path)
    filename = os.path.basename(path)
    if not filename or not filename.endswith('.pdf'):
        filename = f"notice_{index+1}.pdf"
    return f"{index+1}_{filename}"

# Keep track of any downloaded URLs to avoid duplicates
downloaded_urls = set()

# ------------------------------------------------------------------
# 3) Find how many pages of notices exist
#    We look for all pagination buttons:
#       <button type="button" data-testid="pagination-page-number" aria-label="Page 2">2</button>
# ------------------------------------------------------------------
pagination_buttons = driver.find_elements(By.XPATH, "//button[@data-testid='pagination-page-number']")
if not pagination_buttons:
    print("No pagination buttons found. Maybe only 1 page. We'll just scrape whatever is here.")
    total_pages = 1
else:
    # The last button should have the highest page number
    last_label = pagination_buttons[-1].get_attribute('aria-label')  # e.g. "Page 4"
    total_pages = int(last_label.replace("Page ", ""))

print(f"Detected {total_pages} total pages.")

# ------------------------------------------------------------------
# 4) Loop through each page, click it, scrape, download
# ------------------------------------------------------------------
for page_num in range(1, total_pages + 1):
    print(f"\n--- Navigating to page {page_num} ---")

    # (Re)locate the button for "Page X" each time to avoid stale references
    page_btn_locator = (By.XPATH, f"//button[@data-testid='pagination-page-number' and @aria-label='Page {page_num}']")
    
    # If there's only 1 page, or no pagination button, skip trying to click
    if total_pages > 1:
        try:
            page_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(page_btn_locator)
            )
            page_btn.click()
            time.sleep(3)  # Wait for the table to load
        except Exception as e:
            print(f"Could not click page {page_num} button. Error: {e}")
            # If we can't click, we stop or break
            break

    # ------------------------------------------------------------------
    # Now do your scraping & PDF download logic for this page
    # Locate the table rows & eye buttons
    # ------------------------------------------------------------------
    try:
        # Get the rows first (for naming or other info)
        table_rows = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table.usa-table tbody tr"))
        )
        print(f"Found {len(table_rows)} rows on page {page_num}.")

        # Locate the "eye" buttons (one per row, presumably)
        eye_buttons = driver.find_elements(By.CSS_SELECTOR, "button.view-notice")
        print(f"Found {len(eye_buttons)} 'eye' buttons on page {page_num}.")

        for index, btn in enumerate(eye_buttons):
            print(f"\nProcessing row {index+1}/{len(eye_buttons)} on page {page_num}")

            # Grab row info to help name the file, if desired
            notice_id = f"notice_{index+1}"
            try:
                if index < len(table_rows):
                    row = table_rows[index]
                    cells = row.find_elements(By.TAG_NAME, "td")
                    if cells:
                        # Use one of the cell's text as an ID
                        raw_id = cells[0].text.strip()  # e.g. the first cell
                        # Clean up invalid filename chars
                        notice_id = re.sub(r'[\\/*?:"<>|]', "_", raw_id) or notice_id
            except Exception as e:
                print(f"Error extracting row data: {e}")

            # Scroll the button into view, then click
            driver.execute_script("arguments[0].scrollIntoView(true);", btn)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", btn)
            print(f"Clicked 'eye' button for row {index+1}")
            time.sleep(2)

            # Clean up any extra tabs
            while len(driver.window_handles) > 1:
                driver.switch_to.window(driver.window_handles[1])
                driver.close()
                driver.switch_to.window(driver.window_handles[0])

            # Find PDF links in the popup
            try:
                link_elements = WebDriverWait(driver, 3).until(
                    EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, '.pdf')]"))
                )
                print(f"Found {len(link_elements)} PDF link(s) in popup.")

                for i, link in enumerate(link_elements):
                    pdf_href = link.get_attribute("href")

                    if pdf_href in downloaded_urls:
                        print(f"Skipping already downloaded PDF: {pdf_href}")
                        continue

                    # Get a clean filename
                    filename = get_filename_from_url(pdf_href, index)

                    # Track the files in our directory before download
                    before_files = set(os.listdir(download_dir))

                    print(f"Opening PDF link in new tab: {pdf_href}")
                    driver.execute_script(f"window.open('{pdf_href}');")

                    downloaded_urls.add(pdf_href)

                    # Wait up to ~30 seconds for the new file to appear
                    download_complete = False
                    for _ in range(30):
                        time.sleep(1)
                        current_files = set(os.listdir(download_dir))
                        new_files = current_files - before_files
                        if new_files:
                            print(f"New files detected: {new_files}")
                            download_complete = True
                            if len(new_files) == 1:
                                downloaded_file = os.path.join(download_dir, list(new_files)[0])
                                new_file_path = os.path.join(download_dir, f"{notice_id}_{filename}")
                                try:
                                    time.sleep(1)  # small extra delay
                                    os.rename(downloaded_file, new_file_path)
                                    print(f"Renamed downloaded file to: {new_file_path}")
                                except Exception as e:
                                    print(f"Error renaming file: {e}")
                            break

                    # Close extra tabs
                    while len(driver.window_handles) > 1:
                        driver.switch_to.window(driver.window_handles[1])
                        driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                    if not download_complete:
                        print("Download may not have completed within the timeout.")

                    # If you only want to download the first PDF link per row, break here:
                    # break

            except Exception as e:
                print(f"No PDF links found or error reading them: {e}")

            # Attempt to close the popup
            try:
                close_button = WebDriverWait(driver, 2).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 
                        "button.usa-button.usa-modal__close[aria-label='Close this window'][data-close-modal='true']"))
                )
                driver.execute_script("arguments[0].click();", close_button)
                print("Closed popup (exact close button).")
            except Exception as e:
                # Fallback attempt
                print(f"Error with exact close button: {e}")
                try:
                    alt_closes = driver.find_elements(By.XPATH,
                        "//button[contains(@class, 'usa-modal__close') or @aria-label='Close this window' or @data-close-modal='true']")
                    if alt_closes:
                        driver.execute_script("arguments[0].click();", alt_closes[0])
                        print("Closed popup (alternative close button).")
                    else:
                        from selenium.webdriver.common.keys import Keys
                        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                        print("Closed popup via ESC key.")
                except Exception as e2:
                    print(f"All close attempts failed: {e2}")

            time.sleep(2)  # short pause between each row

    except Exception as ex:
        print(f"Error scraping/downloading on page {page_num}: {ex}")
        # Optionally continue or break

# ------------------------------------------------------------------
# 5) Cleanup
# ------------------------------------------------------------------
print("All done. Closing browser.")
kill_browser()


Loaded initial page
Clicked 'I Accept'
Could not click 'Table View': Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7F87A4C25+3179557]
	(No symbol) [0x00007FF7F84088A0]
	(No symbol) [0x00007FF7F82991CA]
	(No symbol) [0x00007FF7F82EFA67]
	(No symbol) [0x00007FF7F82EFC9C]
	(No symbol) [0x00007FF7F8343627]
	(No symbol) [0x00007FF7F8317C6F]
	(No symbol) [0x00007FF7F83402F3]
	(No symbol) [0x00007FF7F8317A03]
	(No symbol) [0x00007FF7F82E06D0]
	(No symbol) [0x00007FF7F82E1983]
	GetHandleVerifier [0x00007FF7F88067CD+3579853]
	GetHandleVerifier [0x00007FF7F881D1D2+3672530]
	GetHandleVerifier [0x00007FF7F8812153+3627347]
	GetHandleVerifier [0x00007FF7F857092A+868650]
	(No symbol) [0x00007FF7F8412FFF]
	(No symbol) [0x00007FF7F840F4A4]
	(No symbol) [0x00007FF7F840F646]
	(No symbol) [0x00007FF7F83FEAA9]
	BaseThreadInitThunk [0x00007FFB13FBE8D7+23]
	RtlUserThreadStart [0x00007FFB1427BF6C+44]

Detected 14 total pages.

--- Navigating to page 1 ---
Found 10 rows on page 1.
Found 10 'eye' buttons o